In [1]:
!git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 810 (delta 202), reused 281 (delta 119), pack-reused 431
Receiving objects: 100% (810/810), 8.42 MiB | 10.55 MiB/s, done.
Resolving deltas: 100% (478/478), done.


In [2]:
%cd PyTorch-Architectures/modeling_gpt2/

/kaggle/working/PyTorch-Architectures/modeling_gpt2


In [3]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 147 kB 820 kB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 21.4 MB/s eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.16.0
    Uninstalling pyarrow-0.16.0:
      Successfully uninstalled pyarrow-0.16.0


In [4]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import GPT2Tokenizer
from model import GPT2Classify
from utils import Conv1D # init_weights throws an error

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = load_dataset('imdb')
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = '[PAD]'

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


In [5]:
def init_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
model = GPT2Classify()
model.apply(init_weights)
model.to(device)

GPT2Classify(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Con

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  60650496


In [7]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, list_texts, list_labels, max_seq_length=256):
        self.tokenizer = tokenizer
        self.list_texts = list_texts
        self.list_labels = list_labels
        self.max_seq_length = max_seq_length
        self.list_train = []
        self.build()
    
    def __len__(self):
        return len(self.list_train)
    
    def __getitem__(self, index):
        input_ids = self.list_train[index]['input_ids']
        attention_mask = self.list_train[index]['attention_mask']
        label = self.list_train[index]['label']
        return {
            'ids': torch.tensor(input_ids, dtype=torch.long),
            'mask': torch.tensor(input_ids, dtype=torch.long),
            'label': torch.tensor(label),
        }
    
    def build(self):
        for text, label in zip(self.list_texts, self.list_labels):
            tokens = self.tokenizer(text, max_length=self.max_seq_length, padding='max_length', truncation=True)
            self.list_train.append({'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask'], 'label': label})

In [8]:
texts_train = dataset['train']['text'][:22500]
labels_train = dataset['train']['label'][:22500]

texts_valid = dataset['train']['text'][22500:]
labels_valid = dataset['train']['label'][22500:]

In [9]:
train_dataset = TextDataset(tokenizer, texts_train, labels_train)
valid_dataset = TextDataset(tokenizer, texts_valid, labels_valid)

In [10]:
batch_size = 16
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size * 2)
test_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size * 2)

print("Length of Train loader: ", len(train_loader))
print("Length of Valid loader: ", len(valid_loader))
print("Length of Test loader: ", len(test_loader))

Length of Train loader:  1407
Length of Valid loader:  79
Length of Test loader:  79


In [11]:
# Check Train Loader:
for sample in train_loader:
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    labels = sample['label'].to(device)
    print(ids.shape, mask.shape, labels.shape)
    break

torch.Size([16, 256]) torch.Size([16, 256]) torch.Size([16])


In [12]:
learning_rate = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [13]:
EPOCHS = 3

def compute_accuracy(model, data_loader, device):
    correct, total = 0, 0
    model.eval()
    with torch.set_grad_enabled(False):
        for sample in data_loader:
            ids = sample['ids'].to(device)
            mask = sample['mask'].to(device)
            labels = sample['label'].to(device)
            outputs = model(input_ids=ids, attention_mask=mask)
            logits = outputs[0]
            probas = F.softmax(logits, dim=-1)
            _, predicted_labels = torch.max(probas, 1)
            correct += (predicted_labels == labels).sum()
            total += labels.size(0)
    return correct.float() / total * 100
        
start_time = time.time()
for epoch in range(EPOCHS):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
        ids = sample['ids'].to(device)
        mask = sample['mask'].to(device)
        labels = sample['label'].to(device)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # LOGGING
        if batch_idx % 200 == 0:
            print("Batch: %04d/%04d || Epoch: %04d/%04d" % (batch_idx, len(train_loader), epoch+1, EPOCHS))
    
    model.eval()
    with torch.set_grad_enabled(False):
        train_accuracy = compute_accuracy(model, train_loader, device)
        valid_accuracy = compute_accuracy(model, valid_loader, device)
        print("Train Accuracy: %.2f || Valid Accuracy: %.2f" % (train_accuracy, valid_accuracy))
    elapsed_time = (time.time() - start_time) / 60
    print("Epoch Elapsed Time: ", elapsed_time)
elapsed_time = (time.time() - start_time) / 60
print("Total Training Time: ", elapsed_time)

Batch: 0000/1407 || Epoch: 0001/0003
Batch: 0200/1407 || Epoch: 0001/0003
Batch: 0400/1407 || Epoch: 0001/0003
Batch: 0600/1407 || Epoch: 0001/0003
Batch: 0800/1407 || Epoch: 0001/0003
Batch: 1000/1407 || Epoch: 0001/0003
Batch: 1200/1407 || Epoch: 0001/0003
Batch: 1400/1407 || Epoch: 0001/0003
Train Accuracy: 55.68 || Valid Accuracy: 0.04
Epoch Elapsed Time:  3.980141774813334
Batch: 0000/1407 || Epoch: 0002/0003
Batch: 0200/1407 || Epoch: 0002/0003
Batch: 0400/1407 || Epoch: 0002/0003
Batch: 0600/1407 || Epoch: 0002/0003
Batch: 0800/1407 || Epoch: 0002/0003
Batch: 1000/1407 || Epoch: 0002/0003
Batch: 1200/1407 || Epoch: 0002/0003
Batch: 1400/1407 || Epoch: 0002/0003
Train Accuracy: 80.66 || Valid Accuracy: 40.44
Epoch Elapsed Time:  7.946165398756663
Batch: 0000/1407 || Epoch: 0003/0003
Batch: 0200/1407 || Epoch: 0003/0003
Batch: 0400/1407 || Epoch: 0003/0003
Batch: 0600/1407 || Epoch: 0003/0003
Batch: 0800/1407 || Epoch: 0003/0003
Batch: 1000/1407 || Epoch: 0003/0003
Batch: 1200/140